In [1]:
import argparse
import datetime
import re
import zipfile

import pandas as pd
import torch
from databench_eval import Evaluator, Runner, utils
from datasets import Dataset
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

from src.code_fixer import CodeFixer
from src.column_selector import ColumnSelector

qa = utils.load_qa(name="semeval", split="dev")

/home/roi/miniconda3/envs/tabular/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.DataFrame(qa)

# Access 150th row
row = df.iloc[150]
spain = utils.load_table(row["dataset"])

spain

,Vote Intention,Age,const,edu,gender,lang,partyId,polInterest,prevVote,prevVoteCat,...,It should be more difficult for companies to lay off workers,Bailing out banks is needed in order to restore economic stability,Increasing university fees is necessary for the sustainability of the university system,The government should subsidize clean energy in order to encourage its use,The subject of religion should be part of the curriculum,The decriminalizati on of soft drugs for personal use is a positive measure,User self-placement on Left-Right economic values axis,User self- placement on Progressive-Conservative economic values axis,Constitutional organisation of the country (More Descentralization),voteReason
0,[PP],105.0,Rest of Spain,I prefer not to say,I prefer not to say,Castellano,Podemos,I prefer not to say,PSOE,NaN,...,4.0,1.0,2.0,1.0,1.0,3.0,5.0,5.0,NaN,The party helps people like me
1,[I will not vote],105.0,Rest of Spain,Not completed Primary education,Female,Castellano,NaN,Not interested at all,I did not vote,NaN,...,1.0,2.0,4.0,5.0,2.0,1.0,10.0,10.0,1.0,NoVote
2,[I am undeceided],89.0,Rest of Spain,Not completed Primary education,Female,Castellano,NaN,Not interested at all,I did not vote,NaN,...,4.0,3.0,2.0,4.0,4.0,1.0,5.0,5.0,3.0,Other
3,[PP],89.0,Rest of Spain,Not completed Primary education,Female,Castellano,PP,Very interested,NaN,NaN,...,1.0,1.0,1.0,4.0,1.0,1.0,5.0,5.0,1.0,The party ideas are close to my own
4,[],87.0,Catalunya,I prefer not to say,Male,Castellano,NaN,I prefer not to say,NaN,NaN,...,2.0,1.0,4.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,[Ciudadanos],NaN,Rest of Spain,University degree,Male,Castellano,Ciudadanos,Very interested,I did not vote,NaN,...,2.0,4.0,3.0,2.0,2.0,5.0,3.0,7.0,1.0,The party ideas are close to my own
19996,[I prefer not to say],NaN,Catalunya,Postgraduate degree,Male,Castellano,I prefer not to say,I prefer not to say,UPyD,Ciudadanos,...,2.0,1.0,3.0,2.0,1.0,1.0,3.0,2.0,1.0,I prefer not to say
19997,[other],NaN,Rest of Spain,NaN,NaN,Castellano,VOX,NaN,I prefer not to say,NaN,...,4.0,2.0,4.0,1.0,5.0,3.0,10.0,10.0,1.0,The party ideas are close to my own
19998,[],NaN,Rest of Spain,University degree,Male,Castellano,NaN,Very interested,PP,NaN,...,4.0,2.0,2.0,5.0,1.0,3.0,3.0,2.0,4.0,The party ideas are close to my own


In [3]:
df

,question,answer,type,columns_used,column_types,sample_answer,dataset
0,Is the most favorited author mainly communicat...,True,boolean,"['favorites', 'lang']","['category', 'category']",True,050_ING
1,Does the author with the longest name post mai...,True,boolean,"['author_name', 'type']","['category', 'category']",False,050_ING
2,Is there an author who received no retweets fo...,True,boolean,"['author_name', 'retweets']","['category', 'number[uint8]']",True,050_ING
3,Are there any posts that do not contain any li...,True,boolean,['links'],['list[url]'],True,050_ING
4,How many unique authors are in the dataset?,3943,number,['author_name'],['category'],20,050_ING
...,...,...,...,...,...,...,...
315,Which 2 countries made the least transactions?,"['Saudi Arabia', 'Bahrain']",list[category],['Country'],['category'],"['United Kingdom', 'Australia']",065_RFM
316,What are the invoice numbers of the top 5 tran...,"['581483', '541431', '578841', '542504', '5730...",list[number],"['InvoiceNo', 'Quantity']","['number[UInt32]', 'number[int32]']","['548005', '555200', '573399', '563745', '5689...",065_RFM
317,What are the stock codes of the bottom 5 trans...,"[23843, 23166, 23005, 23005, 84347]",list[number],"['InvoiceNo', 'Quantity']","['number[UInt32]', 'number[int32]']","['22465', '21109', '82484', '22799', '82600']",065_RFM
318,What are the four most commonly ordered quanti...,"[1, 2, 12, 6]",list[number],['Quantity'],['number[int32]'],"[1, 4, 2, 12]",065_RFM


In [2]:
import pandas as pd
import re

# Generate a new dataframe with columns "Edad" and "Satisfacción"
# And add 4 rows: 4, Mucha, 5, Poca, 6, Regular, 7, Poca
df = pd.DataFrame(
    {
        "Edad": [4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7],
        "Satisfacción": ["Mucha", "Poca", "Regular", "Poca", "Mucha", "Poca", "Regular", "Poca", "Mucha", "Poca", "Regular", "Poca", "Mucha", "Poca", "Regular", "Poca"],
        "aaa": ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p"],
    }
)

def columns_unique(df, selected_columns):
    
    # selected_columns is given as a string, so we need to convert it to a list of strings
    selected_columns = re.findall(r"'(.*?)'", selected_columns)
    
    # Take the columns of the dataframe that are in the selected_columns list
    df = df[selected_columns]
    
    # Remove all columns that contain anything other than strings
    df = df.select_dtypes(include=["object"])
    
    # Count all the unique values of the columns
    counts = [df[col_name].nunique() for col_name in df.columns]
    # If the column has more than 10 unique values, remove it
    for i, col_name in enumerate(df.columns):
        if counts[i] > 10:
            df = df.drop(col_name, axis=1)
    
    # Print all the unique values of the columns
    result = [df[col_name].unique() for col_name in df.columns]
    
    result_str = ""
    
    for i, col_name in enumerate(df.columns):
        result_str += f"Column {col_name} has the following unique values: {result[i]}\n"
    
    return result_str


result = columns_unique(df, "Index(['Edad', 'Satisfacción', 'aaa'])")

print(result)

Column Satisfacción has the following unique values: ['Mucha' 'Poca' 'Regular']

